In [ ]:
#Import all required libraries
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator

In [ ]:
def load_data_spacy(file_path):
    ''' Converts data from:
    word \t label \n word \t label \n \n word \t label
    to: sentence, {entities : [(start, end, label), (stard, end, label)]}
    '''
    file = open(file_path, 'r')
    training_data, entities, sentence, unique_labels = [], [], [], []
    current_annotation = None
    start =0
    end = 0 # initialize counter to keep track of start and end characters
    for line in file:
        line = line.strip("\n").split("\t")
        # lines with len > 1 are words
        if len(line) > 1:
            label = line[1]
            if(label != 'O'):
                label = line[1]+"_Disease"     # the .txt is formatted: label \t word, label[0:2] = label_type
            #label_type = line[0][0] # beginning of annotations - "B", intermediate - "I"
            word = line[0]
            sentence.append(word)
            start = end
            end += (len(word) + 1)  # length of the word + trailing space
           
            if label == 'I_Disease' :  # if at the end of an annotation
                entities.append(( start,end-1, label))  # append the annotation
                              
            if label == 'B_Disease':                         # if beginning new annotation
                entities.append(( start,end-1, label))# start annotation at beginning of word
                
           
           
            if label != 'O' and label not in unique_labels:
                unique_labels.append(label)
 
        # lines with len == 1 are breaks between sentences
        if len(line) == 1:
            if(len(entities) > 0):
                sentence = " ".join(sentence)
                training_data.append([sentence, {'entities' : entities}])
            # reset the counters and temporary lists
            end = 0 
            start = 0
            entities, sentence = [], []
             
    file.close()
    return training_data, unique_labels   

In [ ]:
#TRAIN_DATA, LABELS = load_data_spacy("RheumoData.tsv")
print(TRAIN_DATA)
print(len(TRAIN_DATA))
#TEST_DATA, _ = load_data_spacy("test.tsv")
#print(len(TEST_DATA))
#VALID_DATA, _ = load_data_spacy("train_dev.tsv")
#print(len(VALID_DATA))

In [ ]:
import pandas as pd
import json
            
with open('annotations.json', 'r') as file:
     TRAIN_DATA = json.load(file)

In [ ]:
TRAIN_DATA

In [ ]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

#for text, annot in tqdm(TRAIN_DATA):
for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object



#################################################################################################################





In [ ]:
Create base config file from here: https://spacy.io/usage/training #quickstart

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

In [ ]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./dev.spacy

In [ ]:
#ner = spacy.load(R"ner_demo/training/model-best") #load the best model



#test_sentences = [x[0] for x in TEST_DATA[0:4000]] # extract the sentences from [sentence, entity]
#for x in test_sentences:
#    doc = ner(x)
#    for ent in doc.ents:
#        print(ent.text, ent.start_char, ent.end_char, ent.label_)
#        displacy.render(doc,jupyter=True, style = "ent")
   

In [ ]:
ner = spacy.load(R"model-best") #load the best model
doc = ner(" Amanda Psyrri, MD-PhD, achieved her MD from Medical School at University of Patras in Greece in 1993. From 1996-2002, Prof. Psyrri did her Residency in Internal Medicine, followed by Medical Oncology/Hematology Fellowship at Yale University School of Medicine & Yale Cancer Center in the USA. She is Board-Certified in Internal Medicine and Medical Oncology in USA and in Greece. Prof. Psyrri also did a research Fellowship in Cancer Virology at Professor Daniel DiMaio's laboratory at Yale University (2000-2002). She joined the Faculty of Medicine at Section of Medical Oncology at Yale as Instructor in 2002 and was promoted to Assistant Professor in 2003. At Yale, Prof. Psyrri led a clinical and translational research program in head and neck malignancies focusing on HPV-induced carcinogenesis and biomarkers. Prof. Psyrri's laboratory contributed to the field by providing laboratory evidence for the causal association between HPV and oropharyngeal carcinoma. Based on her laboratory work and others' work, p16 was established as surrogate biomarker for biologically and clinically relevant HPV infection in oropharynx cancer. She was laboratory chair of the ECOG E2303 protocol and demonstrated the association between RAS/PI3K pathway and resistance to cetuximab in head and neck squamous cell carcinoma (HNSCC). Her work highlighted the role of HRAS mutations in head and neck carcinogenesis, which subsequently became therapeutically relevant, as shown by initial encouraging results of tipifarnib in HRAS mutant HNSCC. In 2007, Prof. Psyrri received her Ph.D. Degree from the National Kapodistrian University of Athens and joined the Faculty of Medicine in 2009. In 2012, Professor Psyrri was appointed Head of Medical Oncology at Attikon University Hospital. Prof. Psyrri is currently in charge of research, teaching, and patient care in medical oncology and head and neck malignancies at the Section of Oncology of Attikon University Hospital. She also leads a Molecular Oncology laboratory. Prof. Psyrri's main field of interest is new biomarker discovery and validation in preclinical and clinical settings. Her current research projects are focused on combined immunotherapy to treat recurrent/metastatic HNSCC and identifying predictors for response to PD1 inhibitors in HNSCC. She is steering committee member of several industry-sponsored and PI of investigator-initiated clinical trials of immunotherapy in HNSCC. Between 2007 and 2015, Prof. Psyrri was Translational Research Chair of the EORTC Head and Neck Cancer Group. She is also the Chair of the Head and Neck Cancer Group of the Hellenic Cooperative Oncology Group (HeCOG) and founding member and European Representative of the Head and Neck Cancer Intergroup (HNCIG). From 2017-2021 Prof. Psyrri was scientific director of HECOG. Prof. Psyrri is member of the ESMO Faculty for Head and Neck Cancer. She has been a member of the Scientific Committee of the Head & Neck cancer track at several ESMO Congresses. In 2015, she was ESMO Chair at the International Conference on Innovative Approaches in Head and Neck Oncology in Nice, France. She was co-Chair of the Scientific Committee Track Head & Neck Cancer for ECCO 2015 in Vienna, Chair of the Scientific Committee Track Head & Neck Cancer of ESMO 2016 and ESMO 2018 congresses, and Co-Chair of ESMO ASIA Head & Neck Cancer 2017 and 2020 Congress. Prof. Psyrri has been invited Faculty Speaker at numerous international oncology conferences including ASCO Annual meetings, ESMO Congresses, American Head &Neck Society Meetings, American Society of Radiation Oncology Meeting, HPV Eurogin Conferences.  She is member of AACR, ASCO, EORTC, and Society of Immunotherapy of Cancer (SITC). She is a founding member of Women for Oncology -Greece and member of SITC's Women in Cancer Immunotherapy Network. Prof. Psyrri has authored numerous peer-reviewed manuscripts, and book chapters, served as associate editor of 'Frontiers in Oncology and became Deputy Editor of Oral Oncology in 2016. She is a member of the Editorial Board of Cancers, Annals of Oncology and Journal of Clinical Oncology ")
displacy.render(doc,jupyter=True, style = "ent")

In [ ]:
db = DocBin()
for text, annot in tqdm(VALID_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./valid.spacy") # save the docbin object

In [1]:
#Import all required libraries
import spacy
import random
import time
import numpy as np
import sys
from spacy import displacy
from itertools import chain
import matplotlib.pyplot as plt 
from matplotlib.ticker import MaxNLocator

In [2]:
import pandas as pd
import json
            
with open('annotations.json', 'r') as file:
     data = json.load(file)

In [10]:
data

{'classes': ['COLLEGE'],
 'annotations': [["Lee was born in Taiwan and received her bachelor's degree from Barnard College, where she majored in chemistry. \r",
   {'entities': [(63, 70, 'COLLEGE')]}],
  ['She earned her medical degree at the University of Medicine and Dentistry of New Jersey, and completed her residency in Radiation Oncology at the Columbia Presbyterian Medical Center. Lee has two sons and her husband is an academic ENT surgeon. \r',
   {'entities': [(77, 87, 'COLLEGE'), [146, 154, 'COLLEGE']]}],
  None,
  None,
  None,
  None,
  None,
  [' In 2017 the American Society for Radiation Oncology designated her as a fellow.    ',
   {'entities': [[13, 29, 'COLLEGE']]}]]}

In [11]:
#data = retrieve_data()  # Retrieve the data from a source

if data is not None:
    TRAIN_DATA = data['annotations']
    TRAIN_DATA = [tuple(i) for i in TRAIN_DATA]
    # Rest of your code that uses TRAIN_DATA
else:
    print("Error: Unable to retrieve data.")



TypeError: 'NoneType' object is not iterable

In [7]:
entity_name = "COLLEGE"

TRAIN_DATA = data['annotations']
TRAIN_DATA = [tuple(i) for i in TRAIN_DATA]

TypeError: 'NoneType' object is not iterable

In [12]:
for i in TRAIN_DATA:
    if i[1]['entities'] == []:
        i[1]['entities'] = (0, 0, entity_name)
    else:
        i[1]['entities'][0] = tuple(i[1]['entities'][0])

TypeError: 'NoneType' object is not subscriptable